# **Download the required libraries**




In [1]:
!pip install -qU langchain langchain-community pypdf sentence-transformers openai tiktoken chromadb huggingface-hub token google-generativeai langchain-google-genai unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 1

In [2]:
import textwrap
def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text


# **load the data**

In [ ]:
from langchain.document_loaders import DirectoryLoader,PyPDFLoader
loader = DirectoryLoader('data', glob='**/*.pdf', loader_cls=PyPDFLoader)
documents = loader.load()
documents

In [21]:
len(documents)

111

In [22]:
docs = documents[:50]
len(docs)

50

# **Convert into the chunk**

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(docs)

# **Define the openi model and embedding model**

In [27]:
import os
os.environ["OPENAI_API_KEY"]="Enter your api Key"

In [28]:
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [31]:
from langchain.chat_models import ChatOpenAI
model = ChatOpenAI(temperature=0.0, model_name="gpt-3.5-turbo")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


# **Store the data in vector Database**

In [ ]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(texts, embeddings,persist_directory="./chroma_db")

In [33]:
query = "what is mean by attention?"

db.similarity_search(query, k=5)

[Document(page_content='The ﬁrst attention model was proposed by Bah-\ndanau et al. (2014), there are several other types of\nattention proposed, such as the one by Luong et al.', metadata={'page': 4, 'source': 'data/MachineTranslationwithAttention.pdf'}),
 Document(page_content='2016; Vaswani et al., 2017), language modeling (Raffel et al., 2020), speech-to-text (Chan et al.,\n2015; Prabhavalkar et al., 2017; Chiu et al., 2018), etc. Many of these architectures make use of\nattention (Bahdanau et al., 2014), a mechanism that allows the network to focus on a speciﬁc part of\nthe input most relevant to the current prediction step. Attention has proven to be a critical mechanism;\nindeed many modern architectures, such as the Transformer, are fully attention-based (Vaswani et al.,\n2017). However, despite the success of these architectures, an understanding of how said networks\nsolve such tasks using attention remains largely unknown.\nAttention mechanisms are attractive because they ar

# **Define the retriever**

In [35]:
reteriver=db.as_retriever()
reteriver.get_relevant_documents(query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='The ﬁrst attention model was proposed by Bah-\ndanau et al. (2014), there are several other types of\nattention proposed, such as the one by Luong et al.', metadata={'page': 4, 'source': 'data/MachineTranslationwithAttention.pdf'}),
 Document(page_content='2016; Vaswani et al., 2017), language modeling (Raffel et al., 2020), speech-to-text (Chan et al.,\n2015; Prabhavalkar et al., 2017; Chiu et al., 2018), etc. Many of these architectures make use of\nattention (Bahdanau et al., 2014), a mechanism that allows the network to focus on a speciﬁc part of\nthe input most relevant to the current prediction step. Attention has proven to be a critical mechanism;\nindeed many modern architectures, such as the Transformer, are fully attention-based (Vaswani et al.,\n2017). However, despite the success of these architectures, an understanding of how said networks\nsolve such tasks using attention remains largely unknown.\nAttention mechanisms are attractive because they ar

# **Chain**

In [37]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt=ChatPromptTemplate.from_template(template)

chain=(
    {"context":reteriver,"question":RunnablePassthrough() }
    |prompt
    |model
    |StrOutputParser()
)


In [39]:
text_reply = chain.invoke("Describe the attention layer ?")

print(wrap_text(text_reply))

The attention layer is a mechanism that allows the network to focus on a specific part of
the input that is most relevant to the current prediction step. It is a critical mechanism
in many modern architectures, such as the Transformer, which are fully attention-based.
Attention mechanisms are attractive because they are interpretable and often illuminate
key computations required for a task. The dynamics of attention can vary across tasks and
architectures, and understanding how networks form attention and how they solve tasks
using attention remains largely unknown.


# **With Rag Fusion**

In [40]:
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate,ChatPromptTemplate,PromptTemplate

In [41]:
prompt = ChatPromptTemplate(input_variables=['original_query'],
                            messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(
                                input_variables=[],template='You are a helpful assistant that generates multiple search queries based on a single input query.'
                                )),
                            HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['original_query'], template='Generate multiple search queries related to: {question} \n OUTPUT (4 queries):'))])


In [43]:
original_query = "Describe the attention layer ?"

In [44]:
generate_queries = (
    prompt | model | StrOutputParser() | (lambda x: x.split("\n"))
)


In [45]:
generate_queries

ChatPromptTemplate(input_variables=['question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant that generates multiple search queries based on a single input query.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='Generate multiple search queries related to: {question} \n OUTPUT (4 queries):'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7876382601f0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x787638261900>, temperature=0.0, openai_api_key='sk-proj-9BUSUpqoa5eIUjtDXQayT3BlbkFJabpGeJYocUgAC9Sy4xnZ', openai_proxy='')
| StrOutputParser()
| RunnableLambda(lambda x: x.split('\n'))

# **Reciprocal Rank**

In [46]:
from langchain.load import dumps, loads


def reciprocal_rank_fusion(results: list[list], k=60):
    fused_scores = {}
    for docs in results:
        # Assumes the docs are returned in sorted order of relevance
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    return reranked_results

In [47]:
ragfusion_chain = generate_queries | reteriver.map() | reciprocal_rank_fusion

In [48]:
import langchain
langchain.debug = True

In [49]:
ragfusion_chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'question': {'title': 'Question', 'type': 'string'}}}

In [50]:
ragfusion_chain.invoke({"question": original_query})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Describe the attention layer ?"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "question": "Describe the attention layer ?"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are a helpful assistant that generates multiple search queries based on a single input query.\nHuman: Generate multiple search queries related to: Describe the attention layer ? \n OUTPUT (4 queries):"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] [1.50s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "1. What is the purpose of the attention layer in neural networks?\n2. How does the attention layer improve the performance of deep learning models?\n3. E

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


[(Document(page_content='2016; Vaswani et al., 2017), language modeling (Raffel et al., 2020), speech-to-text (Chan et al.,\n2015; Prabhavalkar et al., 2017; Chiu et al., 2018), etc. Many of these architectures make use of\nattention (Bahdanau et al., 2014), a mechanism that allows the network to focus on a speciﬁc part of\nthe input most relevant to the current prediction step. Attention has proven to be a critical mechanism;\nindeed many modern architectures, such as the Transformer, are fully attention-based (Vaswani et al.,\n2017). However, despite the success of these architectures, an understanding of how said networks\nsolve such tasks using attention remains largely unknown.\nAttention mechanisms are attractive because they are interpretable, and often illuminate key com-\nputations required for a task. For example, consider neural machine translation—trained networks\nexhibit attention matrices that align words in the encoder sequence with the appropriate correspond-', metadat

In [53]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

full_rag_fusion_chain = (
    {
        "context": ragfusion_chain,
        "question": RunnablePassthrough()
    }
    | prompt
    | model
    | StrOutputParser()
)

In [54]:
full_rag_fusion_chain.input_schema.schema()

{'title': 'RunnableParallel<context,question>Input',
 'type': 'object',
 'properties': {'question': {'title': 'Question', 'type': 'string'}}}

In [55]:
full_rag_fusion_chain.invoke({"question": "Describe the attention layer ?"})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Describe the attention layer ?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question>] Entering Chain run with input:
{
  "question": "Describe the attention layer ?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "question": "Describe the attention layer ?"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "question": "Describe the attention layer ?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Describe the attention layer ?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnableSequence > prompt:ChatPromptTemplate] Enteri

'The attention layer is a mechanism that allows the network to focus on a specific part of the input that is most relevant to the current prediction step. It is a critical mechanism used in many modern architectures, such as the Transformer, which are fully attention-based. Attention mechanisms are attractive because they are interpretable and often illuminate key computations required for a task. In neural machine translation, trained networks exhibit attention matrices that align words in the encoder sequence with the appropriate corresponding words in the target sentence, making it easier to optimize the overall task.'